In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from pathlib import Path

In [24]:
def combined_mse_cosine_loss(y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    y_true_norm = tf.nn.l2_normalize(y_true, axis=1)
    y_pred_norm = tf.nn.l2_normalize(y_pred, axis=1)
    cosine_loss = 1 - tf.reduce_mean(tf.reduce_sum(y_true_norm * y_pred_norm, axis=1))
    return mse + 0.3 * cosine_loss

In [25]:
model_path = "models/resnet50.keras"
new_spectrogram_path = "spectrogram/test_data/good_luck_babe_30_60_spectrogram_win_length=2048_hop_length=512_n_fft=2048.png"
FEATURE_NAMES = [
    "acousticness", "instrumentalness", "liveness", "speechiness",
    "danceability", "energy", "tempo", "valence"
]

if len(FEATURE_NAMES) != 8:
    raise ValueError("FEATURE_NAMES list must contain exactly 8 names.")

In [26]:
# 1. Load the trained model with the custom loss function
print(f"Loading model from: {model_path}")
# Ensure model_path is a string or Path object correctly pointing to your model
# from pathlib import Path # if you want to use Path objects
# model_path_obj = Path(model_path)
# if not model_path_obj.exists():
#     raise FileNotFoundError(f"Model file not found at {model_path}")

try:
    # It's good practice to clear session in notebooks if re-running model related code
    tf.keras.backend.clear_session()

    model = load_model(
        model_path,
        custom_objects={'combined_mse_cosine_loss': combined_mse_cosine_loss}
    )
    print("Model loaded successfully.")
    model.summary()  # Optional: print model summary
except Exception as e:
    print(f"Error loading model: {e}")
    # In a notebook, you might want to raise the exception to stop execution
    raise

Loading model from: models/resnet50.keras
Model loaded successfully.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 11, 25, 2048)   │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,073,370 (244.42 MB)

 Trainable params: 19,979,528 (76.22 MB)

 Non-trainable params: 4,134,784 (15.77 MB)

 Optimizer params: 39,959,058 (152.43 MB)

In [27]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (int(984 / 3), int(2385 / 3)))
    image = tf.keras.applications.resnet50.preprocess_input(image)
    image = tf.expand_dims(image, axis=0)
    return image

In [28]:
# Load image
image = load_image(new_spectrogram_path)

# Make predictions
print("\nMaking prediction...")
try:
    predictions = model.predict(image)

    # predictions will be a numpy array like [[feat1, feat2, ..., feat8]]
    predicted_features = predictions[0]

    # Print the results
    print(f"\nPredicted Audio Features for {new_spectrogram_path}:")
    if len(predicted_features) == len(FEATURE_NAMES):
        for name, value in zip(FEATURE_NAMES, predicted_features):
            print(f"- {name}: {value:.4f}")
    else:
        print("Warning: Number of predicted features does not match FEATURE_NAMES length.")
        print("Raw predictions:", predicted_features)

except Exception as e:
    print(f"Error during prediction: {e}")
    # In a notebook, you might want to raise the exception
    raise


Making prediction...
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

Predicted Audio Features for spectrogram/test_data/good_luck_babe_30_60_spectrogram_win_length=2048_hop_length=512_n_fft=2048.png:
- acousticness: 0.4143
- instrumentalness: 0.5266
- liveness: 0.6081
- speechiness: 0.7111
- danceability: 0.1378
- energy: 0.0843
- tempo: 0.4323
- valence: 0.4454
